<a href="https://colab.research.google.com/github/s-grade-achievers/We-R-demented/blob/main/alzhimers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/archive.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/train/nonDem1120.jpg  
  inflating: /content/train/nonDem1121.jpg  
  inflating: /content/train/nonDem1122.jpg  
  inflating: /content/train/nonDem1123.jpg  
  inflating: /content/train/nonDem1124.jpg  
  inflating: /content/train/nonDem1125.jpg  
  inflating: /content/train/nonDem1126.jpg  
  inflating: /content/train/nonDem1127.jpg  
  inflating: /content/train/nonDem1128.jpg  
  inflating: /content/train/nonDem1129.jpg  
  inflating: /content/train/nonDem1130.jpg  
  inflating: /content/train/nonDem1131.jpg  
  inflating: /content/train/nonDem1132.jpg  
  inflating: /content/train/nonDem1133.jpg  
  inflating: /content/train/nonDem1134.jpg  
  inflating: /content/train/nonDem1135.jpg  
  inflating: /content/train/nonDem1136.jpg  
  inflating: /content/train/nonDem1137.jpg  
  inflating: /content/train/nonDem1138.jpg  
  inflating: /content/train/nonDem1139.jpg  
  inflating: /content/train/nonDem1140.jpg  
  in

In [ ]:
import os

directory = '/content/train'

files = [file for file in os.listdir(directory) if file.endswith('.jpg')]
mildDem = [i for i in files if i.startswith("milDem")]
moderateDem = [i for i in files if "moderateDim" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "veryMildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))


2560 2560 2560 2560


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Assuming mildDem, moderateDem, nonDem, veryMild are lists containing your image filenames
l = [[mildDem, "0"], [moderateDem, "1"], [nonDem, "2"], [veryMild, "3"]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/content/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])

# Split the dataframe into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2)

datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create a generator for the training data
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="Path",
    y_col="Class",
    target_size=(110, 130),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

# Create a generator for the validation data
val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="Path",
    y_col="Class",
    target_size=(110, 130),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)


Found 8192 validated image filenames belonging to 4 classes.
Found 2048 validated image filenames belonging to 4 classes.


##ACTUAL MODEL

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

def conv_block(filters):
    block = Sequential()
    block.add(Conv2D(filters, 3, activation='relu', padding='same'))
    block.add(Conv2D(filters, 3, activation='relu', padding='same'))
    block.add(BatchNormalization())
    block.add(MaxPooling2D((2, 2)))
    return block

In [ ]:
def dense_block(units, dropout_rate):
    block = Sequential()
    block.add(Dense(units, activation='relu'))
    block.add(BatchNormalization())
    block.add(Dropout(dropout_rate))
    return block

In [ ]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes):
        super(F1Score, self).__init__()
        self.num_classes = num_classes
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'), tf.keras.metrics.AUC(name='auc'), F1Score(num_classes=4)]

In [ ]:
input_shape = (110, 130, 3)

def construct_model():
    model = Sequential([
        Input(shape=input_shape),
        conv_block(32),
        conv_block(64),
        conv_block(128),
        Dropout(0.2),
        conv_block(256),
        Dropout(0.2),
        Flatten(),
        dense_block(14288, 0.7),  # Adjust the units based on the output shape of Flatten layer
        dense_block(256, 0.5),
        dense_block(64, 0.3),
        Dense(4, activation='softmax')
    ], name="cnn_model")
    return model

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('acc') > 0.995:
            print("\nReached accuracy threshold! Terminating training.")
            self.model.stop_training = True

# my_callback = MyCallback()
# CALLBACKS = [my_callback, lr_scheduler]

# history = model.fit(train_generator, epochs=EPOCHS, callbacks=CALLBACKS)


In [ ]:
from keras.optimizers import Adam

def lr_schedule(epoch, learning_rate):
    if epoch < 20:
        return learning_rate
    else:
        if epoch % 2 == 0:
          return learning_rate * 0.95
        else:
          return learning_rate

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)
initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)

In [ ]:
my_callback = MyCallback()
CALLBACKS = [my_callback, lr_scheduler]

EPOCHS = 50
history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS, callbacks=CALLBACKS)


Epoch 1: LearningRateScheduler setting learning rate to 0.0005133419763296843.
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:2723: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


256/256 [==============================] - 55s 213ms/step - loss: 0.2085 - acc: 0.9196 - auc: 0.9915 - f1_score: 0.9189 - val_loss: 0.2185 - val_acc: 0.9165 - val_auc: 0.9907 - val_f1_score: 0.9146 - lr: 5.1334e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0005133419763296843.
Epoch 2/50
256/256 [==============================] - 56s 220ms/step - loss: 0.1950 - acc: 0.9241 - auc: 0.9927 - f1_score: 0.9242 - val_loss: 0.2529 - val_acc: 0.8926 - val_auc: 0.9886 - val_f1_score: 0.8922 - lr: 5.1334e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0005133419763296843.
Epoch 3/50
256/256 [==============================] - 55s 214ms/step - loss: 0.1996 - acc: 0.9230 - auc: 0.9923 - f1_score: 0.9224 - val_loss: 0.3351 - val_acc: 0.8687 - val_auc: 0.9814 - val_f1_score: 0.8686 - lr: 5.1334e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0005133419763296843.
Epoch 4/50
256/256 [==============================] - 56s 220ms/step - loss: 0.1974 - acc: 0.

In [ ]:
import cv2
import keras
import numpy as np
from keras.utils import to_categorical
from sklearn.metrics import classification_report

# Load test images and labels
x_test = []
y_test = []

# Assuming you have the following lists of image filenames for each class
mildDem_test = [i for i in files if i.startswith("MildDemented")]
moderateDem_test = [i for i in files if i.startswith("ModerateDemented")]
nonDem_test = [i for i in files if i.startswith("NonDemented")]
veryMild_test = [i for i in files if i.startswith("VeryMildDemented")]

print("Number of mildDemented images: ", len(mildDem_test))
print("Number of moderateDemented images: ", len(moderateDem_test))
print("Number of NonDemented images: ", len(nonDem_test))
print("Number of VeryMildDemented images: ", len(veryMild_test))

l_test = [[mildDem_test, 0], [moderateDem_test, 1], [nonDem_test, 2], [veryMild_test, 3]]

for i in l_test:
    for j in i[0]:
        path = os.path.join("/content/test", j)
        img = cv2.imread(path)
        img = cv2.resize(img, (130, 110))  # Swap the dimensions here
        img = img / 255.0
        x_test.append(img)
        y_test.append(i[1])  # Convert class label to integer

# Convert lists to numpy arrays
x_test = np.array(x_test)
y_test = np.array(y_test)

# Convert labels to one-hot encoding
y_test = keras.utils.to_categorical(y_test, num_classes=4)

# Ensure that the number of test images is a multiple of 32
if len(x_test) % 32 != 0:
    x_test = x_test[:-(len(x_test) % 32)]
    y_test = y_test[:-(len(y_test) % 32)]


Number of mildDemented images:  179
Number of moderateDemented images:  12
Number of NonDemented images:  640
Number of VeryMildDemented images:  448


In [ ]:
test_loss, test_acc, test_auc, test_f1 = model.evaluate(x_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")
print(f"Test AUC: {test_auc}")
print(f"Test F1 Score: {test_f1}")

# Predict the classes
predictions = model.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)

# Get the true classes
true_classes = np.argmax(y_test, axis=1)

# Print the classification report
class_names = ["mildDemented", "moderateDemented", "NonDemented", "VeryMildDemented"]
print(classification_report(true_classes, predicted_classes, target_names=class_names))

 5/39 [==>...........................] - ETA: 0s - loss: 1.6622 - acc: 0.6313 - auc: 0.8421 - f1_score: 0.6332

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:2723: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


39/39 [==============================] - 1s 21ms/step - loss: 1.1068 - acc: 0.7356 - auc: 0.8990 - f1_score: 0.7372
Test Loss: 1.106765866279602
Test Accuracy: 0.7355769276618958
Test AUC: 0.8989831209182739
Test F1 Score: 0.737223207950592
39/39 [==============================] - 1s 15ms/step
                  precision    recall  f1-score   support

    mildDemented       0.77      0.61      0.69       179
moderateDemented       1.00      0.25      0.40        12
     NonDemented       0.89      0.70      0.78       640
VeryMildDemented       0.60      0.86      0.70       417

        accuracy                           0.74      1248
       macro avg       0.82      0.61      0.64      1248
    weighted avg       0.78      0.74      0.74      1248

